In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py
import tqdm

import warnings
warnings.filterwarnings("ignore")

import regressor, rbfnet_aug
import utils

In [2]:
metadata = pd.read_csv('../data/plasticc/plasticc_train_metadata.csv.gz')
data = pd.read_csv('../data/plasticc/plasticc_train_lightcurves.csv.gz')

data = data[data.detected_bool == 1]

object_ids = np.unique(data.object_id)

passband2name = {0: 'u', 1: 'g', 2: 'r', 3: 'i', 4: 'z', 5: 'y'}
passband2lam  = {0: np.log10(3751.36), 1: np.log10(4741.64), 2: np.log10(6173.23), 
                 3: np.log10(7501.62), 4: np.log10(8679.19), 5: np.log10(9711.53)}

In [3]:
def get_object(data, object_id):
    anobject = data[data.object_id == object_id]
    return anobject

def get_passband(anobject, passband):
    light_curve = anobject[anobject.passband == passband]
    return light_curve

def add_log_lam(anobject):
    passbands = anobject.passband.values
    log_lam = [passband2lam[i] for i in passbands]
    anobject['log_lam'] = log_lam
    return anobject

def compile_obj(t, flux, flux_err, passband):
    obj = pd.DataFrame()
    obj['mjd']      = t
    obj['flux']     = flux
    obj['flux_err'] = flux_err
    obj['passband'] = passband
    return obj

def create_approx_object(anobject, n=1000):
    mjd = anobject['mjd'].values
    dfs = []
    for passband in range(6):
        df = pd.DataFrame()
        df['mjd'] = np.linspace(mjd.min(), mjd.max(), n)
        df['object_id'] = 0
        df['passband'] = passband
        df['flux'] = 0
        df['flux_err'] = 0
        df['detected_bool'] = 1
        dfs.append(df)
    new_object = pd.concat(dfs, axis=0)
    new_object = add_log_lam(new_object)
    return new_object

def is_good(anobject):
    good = 1
    
    # remove all objects with negative flux values
    if anobject['flux'].values.min() < 0:
        good = 0
    
    # keep only objects with at least 10 observations in at least 3 passbands
    count = 0
    for passband in range(6):
        if len(get_passband(anobject, passband)) < 10:
            count += 1
    if count > 3:
        good = 0
        
    # keep only objects without large breaks in observations
    anobject = anobject.sort_values('mjd')
    mjd = anobject['mjd'].values
    if np.diff(mjd, 1).max() > 50:
        good = 0
    
    return good

def plot_light_curves(anobject, title=""):
    anobject = anobject.sort_values('mjd')
    plt.figure(figsize=(9, 4))
    for passband in range(6):
        light_curve = get_passband(anobject, passband)
        plt.plot(light_curve['mjd'].values, light_curve['flux'].values, linewidth=0.5)
        plt.scatter(light_curve['mjd'].values, light_curve['flux'].values, label=passband2name[passband], linewidth=1)
    plt.xlabel('Modified Julian Date', size=14)
    plt.xticks(size=14)
    plt.ylabel('Flux', size=14)
    plt.yticks(size=14)
    plt.grid()
    plt.legend(loc='best', ncol=3, fontsize=14)
    plt.title(title, size=14)

In [4]:
from sklearn.model_selection import train_test_split
good_object_ids = [idx for idx in object_ids if is_good(get_object(data, idx))]

In [5]:
MODEL = 'GP'
gp_report = pd.DataFrame(columns=["ID", 'RMSE', 'MAE', 'RSE', 'RAE', 'MAPE'])

for i in tqdm.tqdm_notebook(good_object_ids):
    
    # get an object
    anobject = get_object(data, i)
    anobject = add_log_lam(anobject)
    
    # train / test split
    anobject_train, anobject_test = train_test_split(anobject, test_size=0.5, random_state=11)
    
    X_train = anobject_train[['mjd', 'log_lam']].values
    X_test  = anobject_test[['mjd', 'log_lam']].values

    y_train = anobject_train['flux'].values
    y_test  = anobject_test['flux'].values

    from sklearn.preprocessing import StandardScaler
    ss = StandardScaler()
    X_train_ss = ss.fit_transform(X_train)
    X_test_ss = ss.transform(X_test)

    # fit a regression model to approximate light curves
    if MODEL == "XGB":
        from xgboost import XGBRegressor
        reg = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, objective='reg:squarederror')
    elif MODEL == "NN":
        reg = regressor.FitNNRegressor(n_hidden=100, n_epochs=100, batch_size=1, lr=0.01, lam=0.01, optimizer='Adam')
    elif MODEL == "Linear":
        from sklearn.linear_model import LinearRegression
        reg = LinearRegression()
    elif MODEL == "GP":
        from sklearn.gaussian_process import GaussianProcessRegressor
        from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, WhiteKernel, ConstantKernel as C
        kernel = C(1.0) * RBF([1.0, 1.0]) + WhiteKernel()
        reg = GaussianProcessRegressor(kernel=kernel, normalize_y=True, n_restarts_optimizer=0, optimizer="fmin_l_bfgs_b", random_state=42)

    reg.fit(X_train_ss, y_train)

    
    # predict flux for unseen observations
    anobject_test_pred = anobject_test.copy()
    anobject_test_pred['flux'] = np.maximum(reg.predict(X_test_ss), 0)
    
    metrics = utils.regression_quality_metrics_report(anobject_test['flux'].values, anobject_test_pred['flux'].values)
    gp_report.loc[len(gp_report), :] = [i] + list(metrics)
np.round(gp_report.mean(), 2)

ID      340898.23
RMSE        25.63
MAE         14.25
RSE          0.24
RAE          0.20
MAPE        24.05
dtype: float64

In [6]:
nn_report = pd.DataFrame(columns=["ID", 'RMSE', 'MAE', 'RSE', 'RAE', 'MAPE'])

for i in tqdm.tqdm_notebook(good_object_ids):
    
    # get an object
    anobject = get_object(data, i)
    anobject = add_log_lam(anobject)
    
    # train / test split
    anobject_train, anobject_test = train_test_split(anobject, test_size=0.5, random_state=11)
    
    X_train = anobject_train[['mjd', 'log_lam']].values
    X_test  = anobject_test[['mjd', 'log_lam']].values

    y_train = anobject_train['flux'].values
    y_test  = anobject_test['flux'].values

    from sklearn.pipeline import make_pipeline
    from sklearn.preprocessing import StandardScaler, PolynomialFeatures

    ss = StandardScaler()
    X_train_ss = ss.fit_transform(X_train)
    X_test_ss = ss.transform(X_test)

    # fit a regression model to approximate light curves
    reg = regressor.FitNNRegressor(n_hidden=300, n_epochs=200, batch_size=200, lr=0.1, lam=0.1, optimizer='Adam', debug=0)

    reg.fit(X_train_ss, y_train)

    
    # predict flux for unseen observations
    anobject_test_pred = anobject_test.copy()
    anobject_test_pred['flux'] = np.maximum(reg.predict(X_test_ss), 0)
    
    metrics = utils.regression_quality_metrics_report(anobject_test['flux'].values, anobject_test_pred['flux'].values)
    nn_report.loc[len(nn_report), :] = [i] + list(metrics)
np.round(nn_report.mean(), 2)

ID      340898.23
RMSE        21.21
MAE         12.41
RSE          0.25
RAE          0.20
MAPE        17.33
dtype: float64

In [7]:
rbf_report = pd.DataFrame(columns=["ID", 'RMSE', 'MAE', 'RSE', 'RAE', 'MAPE'])

for i in tqdm.notebook.tqdm(good_object_ids):
    
    # get an object
    anobject = get_object(data, i)
    
    # train / test split
    anobject_train, anobject_test = train_test_split(anobject, test_size=0.5, random_state=11)    
    
    # fit augmentation model
    model = rbfnet_aug.RBFNetAugmentation(passband2lam, n_hidden=300, lam=0., n_epochs=300, lr=0.01, batch_size=100)
    model.fit(anobject_train['mjd'].values, anobject_train['flux'].values, 
              anobject_train['flux_err'].values, anobject_train['passband'].values)
    
    # predict flux for unseen observations
    flux_pred, flux_err_pred = model.predict(anobject_test['mjd'].values, anobject_test['passband'].values, copy=True)
    anobject_test_pred = compile_obj(anobject_test['mjd'].values, flux_pred, 
                                     flux_err_pred, anobject_test['passband'].values)
    metrics = utils.regression_quality_metrics_report(anobject_test['flux'].values, 
                                                      anobject_test_pred['flux'].values)
    rbf_report.loc[len(rbf_report), :] = [i] + list(metrics)
np.round(rbf_report.mean(), 2)

ID      340898.23
RMSE        24.05
MAE         13.20
RSE          0.26
RAE          0.20
MAPE        21.50
dtype: float64

In [8]:
from scipy.stats import wilcoxon

def bootstrap_estimate_mean_stddev(arr, n_samples=10000):
    arr = np.array(arr)
    bs_samples = np.random.randint(0, len(arr), size=(n_samples, len(arr)))
    bs_samples = arr[bs_samples].mean(axis=1)
    sigma = np.sqrt(np.sum((bs_samples - bs_samples.mean())**2) / (n_samples - 1))
    return sigma

def wilcoxon_pvalue(arr1, arr2):
    return wilcoxon(arr1, arr2).pvalue

In [9]:
reports = [gp_report.drop('ID', axis=1),
           nn_report.drop('ID', axis=1),
           rbf_report.drop('ID', axis=1)]

names = ['GP', 'NN', 'RBF-NN']
columns = ['RMSE', 'MAE', 'RSE', 'RAE', 'MAPE']

for i in range(3):
    print(f'Metrics for {names[i]}:')
    for col in columns:
        print(
            col, reports[i][col].values.mean(),
            bootstrap_estimate_mean_stddev(reports[i][col].values)
        )
    print()

for i in range(3):
    for j in range(3):
        if i >= j:
            continue
        print(f'p-values for {names[i]} and {names[j]}:')
        for col in columns:
            pval = wilcoxon_pvalue(reports[i][col], reports[j][col])
            print(col, pval)
        print()

Metrics for GP:
RMSE 25.634116604170423 7.1947785501488575
MAE 14.251705022145114 2.8571275425041107
RSE 0.24219130298576624 0.0065613634281743724
RAE 0.19893157397694908 0.004838990906652575
MAPE 24.046133457430784 0.8938127174865704

Metrics for NN:
RMSE 21.205725934744326 2.0011275920135323
MAE 12.414494605048738 0.9879031474419907
RSE 0.24596566403507372 0.006186012954201068
RAE 0.19589340179161066 0.004422449915219753
MAPE 17.33137969500921 0.5554812260187847

Metrics for RBF-NN:
RMSE 24.05000184900654 4.434749462580905
MAE 13.196447699363711 2.2772838830344124
RSE 0.26159485162812607 0.00721724338645648
RAE 0.20122066839035116 0.005203010341891302
MAPE 21.498071616272448 0.881985642713811

p-values for GP and NN:
RMSE 0.02886710264518012
MAE 0.6223375170117899
RSE 0.23455500723305767
RAE 0.21946322240155725
MAPE 3.616705834635243e-27

p-values for GP and RBF-NN:
RMSE 0.0023317232104010694
MAE 0.5072573365515203
RSE 0.00011487867416114932
RAE 0.7425650192486195
MAPE 8.727055407764